# Purpose


This notebook demonstrates the data pipeline from raw tables to analytical datasets. At the end of this activity, train & test data sets are created from raw data.



## Imports

In [1]:
from pprint import pprint
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, get_package_path,
    display_as_tabs, string_cleaning, merge_info, initialize_environment,
    list_datasets, load_dataset, save_dataset
)
import ta_lib.eda.api as eda

: 

In [4]:
import warnings

warnings.filterwarnings('ignore', message="The default value of regex will change from True to False in a future version.", 
                        category=FutureWarning)

In [5]:
initialize_environment(debug=False, hide_warnings=True)

## Utility functions

# 1. Initialization

In [6]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
pprint(list_datasets(context))

housing_df = load_dataset(context, 'raw/housing')

['/raw/housing',
 '/cleaned/housing',
 '/processed/housing',
 '/train/housing/features',
 '/train/housing/target',
 '/test/housing/features',
 '/test/housing/target',
 '/score/housing/output']


# 2. Data cleaning and consolidation

**<u>NOTES</u>**

The focus here is to create a cleaned dataset that is appropriate for solving the DS problem at hand from the raw data.

**1. Do**
* clean dataframe column names
* ensure dtypes are set properly
* join with other tables etc to create features
* transform, if appropriate, datetime like columns to generate additional features (weekday etc)
* transform, if appropriate, string columns to generate additional features
* discard cols that are not useful for training the model (IDs, constant cols, duplicate cols etc)
* additional features generated from existing columns


**2. Don't**
* handle missing values or outliers here. mark them and leave them for processing downstream.


## 2.1 Clean individual tables 

### Housing Table

* This will go into production code


In [7]:
housing_df_clean = (
    housing_df
    # while iterating on testing, it's good to copy the dataset(or a subset)
    # as the following steps will mutate the input dataframe. The copy should be
    # removed in the production code to avoid introducing perf. bottlenecks.
    .copy()

    # set dtypes : nothing to do here
    .passthrough()
)
housing_df_clean.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### NOTE

It's always a good idea to save cleaned tabular data using a storage format that supports the following 

1. preserves the type information
2. language agnostic storage format
3. Supports compression
4. Supports customizing storage to optimize different data access patterns

For larger datasets, the last two points become crucial.

`Parquet` is one such file format that is very popular for storing tabular data. It has some nice properties:
- Similar to pickles & RDS datasets, but compatible with all languages
- Preserves the datatypes
- Compresses the data and reduces the filesize
- Good library support in Python and other languages
- As a columnar storage we can efficiently read fewer columns
- It also supports chunking data by groups of columns (for instance, by dates or a particular value of a key column) that makes loading subsets of the data fast.

In [8]:
save_dataset(context, housing_df_clean, 'cleaned/housing')

## 2.3 Business intuition features

This section can go into production code if these features are used in final model

In [9]:
housing_df["rooms_per_household"] = housing_df["total_rooms"] / housing_df["households"]
housing_df["bedrooms_per_room"] = housing_df["total_bedrooms"] / housing_df["total_rooms"]
housing_df["population_per_household"] = housing_df["population"] / housing_df["households"]

In [10]:
# create a sample dataframe with minimal processing

housing_df_processed = (
    housing_df
    
    # tweak to test pipeline quickly or profile performance
    #.sample(frac=1, replace=False)
    
    # any additional processing/cleaning
)

# Any verifications on the data
from ta_lib.eda.api import get_variable_summary
display_as_tabs([
    ("Summary", f"Length: {len(housing_df_processed)}, Columns: {len(housing_df_processed.columns)}"),
    ("Variable summary", get_variable_summary(housing_df_processed)),
    ("head", housing_df.head(5).T),
    ("tail", housing_df.tail(5).T),
])

Tabs
    [0] Markdown(str, name='Summary')
    [1] DataFrame(height=300, name='Variable summary', sizing_mode='fixed', value=                          ..., width=800)
    [2] DataFrame(height=300, name='head', sizing_mode='fixed', value=              ..., width=800)
    [3] DataFrame(height=300, name='tail', sizing_mode='fixed', value=              ..., width=800)

In [11]:
save_dataset(context, housing_df_processed, 'processed/housing')

# 3. Generate Train, Validation and Test datasets



- We split the data into train, test (optionally, also a validation dataset)
- In this example, we are binning the median_income into 5 bins and then use a Stratified Shuffle to split the data.
- See sklearn documentation on the various available splitters
- https://scikit-learn.org/stable/modules/classes.html#splitter-classes
- This will go into production code (training only)

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit
from ta_lib.core.api import custom_train_test_split  # helper function to customize splitting
from scripts import *

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
housing_df_train, housing_df_test = custom_train_test_split(housing_df_processed, splitter, by=binned_income)

for set_ in (housing_df_train, housing_df_test):
        set_.drop("income_cat", axis=1, inplace=True)

In [13]:
target_col = "median_house_value"

train_X, train_y = (
    housing_df_train
    
    # split the dataset to train and test
    .get_features_targets(target_column_names=target_col)
)
save_dataset(context, train_X, 'train/housing/features')
save_dataset(context, train_y, 'train/housing/target')


test_X, test_y = (
    housing_df_test
    
    # split the dataset to train and test
    .get_features_targets(target_column_names=target_col)
)
save_dataset(context, test_X, 'test/housing/features')
save_dataset(context, test_y, 'test/housing/target')

### Custom Transformer that supports both forward and inverse transforms

In [14]:
from custom_transformer import CustomTransformer
import unit_test

In [15]:
custom_scaler = CustomTransformer()

In [16]:
num_columns = train_X.select_dtypes('number').columns
train_X_transformed = custom_scaler.transform(train_X[num_columns])
train_X_transformed.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
12655,-0.941322,1.347397,0.027563,0.584760,0.635103,0.732580,0.556269,-0.893620,0.017395,-0.123552,0.006222
15502,1.171747,-1.192404,-1.721965,1.261428,0.775653,0.533595,0.721296,1.292129,0.569238,-0.909033,-0.040810
2908,0.267573,-0.125968,1.220423,-0.469759,-0.545029,-0.674655,-0.524391,-0.525418,-0.018024,-0.370843,-0.075369
14053,1.221701,-1.351433,-0.370057,-0.348641,-0.038565,-0.467603,-0.037296,-0.865903,-0.595122,1.109002,-0.106800
20496,0.437418,-0.635799,-0.131485,0.427167,0.269190,0.374049,0.220892,0.325742,0.251234,-0.525994,0.006109


In [17]:
train_X_inv_transformed = custom_scaler.inverse_transform(train_X,train_X_transformed)
train_X_inv_transformed.head()

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,5.485836,0.205784,3.168555
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,6.927083,0.160714,2.623698
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,5.393333,0.191595,2.223333
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,3.886128,0.276505,1.859213
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,6.096552,0.182692,3.167241
